# Check quote attribution output to see if captured last lines

In [5]:
import os, json
from IPython.core.debugger import set_trace

quote_attribution_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/output/quote_attribution/'

for fname in sorted(os.listdir(quote_attribution_dirpath)):
    with open(os.path.join(quote_attribution_dirpath, fname), 'rb') as f:
        quotes = json.load(f)
        
    last_quote = quotes[-1]
    set_trace()

> <ipython-input-5-a86726d54b3e>(6)<module>()
      4 quote_attribution_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/output/quote_attribution/'
      5 
----> 6 for fname in sorted(os.listdir(quote_attribution_dirpath)):
      7     with open(os.path.join(quote_attribution_dirpath, fname), 'rb') as f:
      8         quotes = json.load(f)

ipdb> p last_quote
{'speaker': 'a_concept_evander', 'quotes': [{'start': 7421, 'end': 7429, 'quote': '" So , what \'s your hypothesis ? "'}], 'paragraph': 234, 'type': 'Implicit', 'start': 7421, 'end': 7493, 'replyto': -1}
ipdb> p fic_id
'999797'
ipdb> p fname
'10001006.quote.json'
ipdb> q


BdbQuit: 

# Put in newlines in char_coref_stories to fix

In [2]:
import re
from tqdm import tqdm_notebook as tqdm
import os

coref_old_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/output/char_coref_stories_old/'
coref_new_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/output/char_coref_stories/'

for fname in tqdm(sorted(os.listdir(coref_old_dirpath))):
    
    fic_id = fname.split('.')[0]
    
    with open(os.path.join(coref_old_dirpath, fname)) as f:
        file = f.read()
        
        # Insert newlines before fic_id
        newtext = file.replace(fic_id, f'\n"\n{fic_id}')
#         set_trace()
        newtext = newtext.replace('\n"\n\n"\n"', '\n"\n')
        
    with open(os.path.join(coref_new_dirpath, fname), 'w') as f:
        f.write(newtext)

# Extract who's talking to who from quote attribution output

In [15]:
# Less restrictive extraction: if other character appears in quote or paragraph before or after
from IPython.core.debugger import set_trace

import os
import json
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict

quote_attribution_output_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/output/quote_attribution/'

pairings = [
    ('draco', 'harry'),
    ('draco', 'hermione'),
    ('ginny', 'harry'),
    ('harry', 'hermione'),
    ('harry', 'ron'),
    ('hermione', 'ron'),
]

fic_contexts = []

for fname in tqdm(sorted(os.listdir(quote_attribution_output_dirpath))[1:2]):
    fic_id = int(fname.split('.')[0])
    with open(os.path.join(quote_attribution_output_dirpath, fname), 'rb') as f:
        quotes = json.load(f)

    speakers = set([s['speaker'].lower() for s in quotes])
    
    for pairing in pairings:
        if any([pairing[0] in s for s in speakers]) and any([pairing[1] in s for s in speakers]):

            # Grab all paragraphs from that speaker
            for c in pairing:
                speaker = [s for s in speakers if c in s]
                other_char = [other for other in pairing if other!=c][0]
                for s in speaker: # could have multiple corresponding speakers
                    speaker_quotes = [q for q in quotes if q['speaker']==s]
                    speaker_replies = [q for q in speaker_quotes if q['replyto'] != -1]

                    # Check reply to what quote
                    for quote in speaker_replies:
                        replied_para = quote['replyto']
                        replied_to = [q for q in quotes if q['paragraph']==replied_para][0]
                        if replied_to['speaker'] == other_char:
                            para_quotes = ' '.join([t['quote'].lower() for t in quote['quotes']])
                            replied_to_quotes = ' '.join([t['quote'].lower() for t in replied_to['quotes']])
                            pairing_quote = ' '.join([para_quotes, replied_to_quotes])
                            fic_contexts.extend(pairing_quote.split())
                            
                    # Check if other is in quote
                    for quote in speaker_quotes:
                        para_quotes = ' '.join([t['quote'].lower() for t in quote['quotes']])
                        if other_char in para_quotes:
                            fic_contexts.extend(para_quotes.split())
                            
                        # Check if other is in following or previous paragraph
                        set_trace()
                        para_id = quote['paragraph']

print(fic_id)
print(c)
print(para_quotes)
print(para_id)

> <ipython-input-15-2289941e20be>(58)<module>()
     56                         # Check if other is in following or previous paragraph
     57                         set_trace()
---> 58                         para_id = quote['paragraph']
     59 
     60 print(fic_id)

ipdb> p quote
{'speaker': 'draco_tense', 'quotes': [{'start': 1298, 'end': 1301, 'quote': "`` ccc_Draco_tense_ccc ! ''"}], 'paragraph': 31, 'type': 'Explicit', 'start': 1298, 'end': 1340, 'replyto': 29}
ipdb> p fic_id
1001234
ipdb> c
> <ipython-input-15-2289941e20be>(57)<module>()
     55 
     56                         # Check if other is in following or previous paragraph
---> 57                         set_trace()
     58                         para_id = quote['paragraph']
     59 

ipdb> p quote
{'speaker': 'draco_tense', 'quotes': [{'start': 1363, 'end': 1381, 'quote': "`` Are you fucking kidding me ? You kissed me ? While I was in a coma ? ''"}], 'paragraph': 33, 'type': 'Explicit', 'start': 1363, 'end': 1385, 

BdbQuit: 

In [6]:
s

'younger_harry'

In [1]:
# Extract from direct quotes and replies

import os
import json
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict

quote_attribution_output_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/output/quote_attribution/'

pairings = [
    ('draco', 'harry'),
    ('draco', 'hermione'),
    ('ginny', 'harry'),
    ('harry', 'hermione'),
    ('harry', 'ron'),
    ('hermione', 'ron'),
]

pairing_quotes = defaultdict(lambda: defaultdict(list))

for fname in tqdm(sorted(os.listdir(quote_attribution_output_dirpath))):
    fic_id = int(fname.split('.')[0])
    with open(os.path.join(quote_attribution_output_dirpath, fname), 'rb') as f:
        quotes = json.load(f)

    speakers = set([s['speaker'].lower() for s in quotes])
    
    for pairing in pairings:
        if any([pairing[0] in s for s in speakers]) and any([pairing[1] in s for s in speakers]):

            # Grab all paragraphs from that speaker, see if replying to the other
            for c in pairing:
                speaker = [s for s in speakers if c in s]
                for s in speaker: # could have multiple corresponding speakers
                    speaker_quotes = [q for q in quotes if q['speaker']==s]
                    speaker_replies = [q for q in speaker_quotes if q['replyto'] != -1]

                    # Check reply to what quote
                    for quote in speaker_replies:
                        replied_para = quote['replyto']
                        replied_to = [q for q in quotes if q['paragraph']==replied_para][0]
                        other_char = [other for other in pairing if other!=c][0]
                        if replied_to['speaker'] == other_char:
                            para_quotes = ' '.join([t['quote'] for t in quote['quotes']])
                            replied_to_quotes = ' '.join([t['quote'] for t in replied_to['quotes']])
                            pairing_quote = ' '.join([para_quotes, replied_to_quotes])
                            pairing_quotes[fic_id][pairing].append(pairing_quote)

# print(speaker_quotes)
# print(speaker_replies)
print(len(pairing_quotes))


1057


In [2]:
pairing_quotes

defaultdict(<function __main__.<lambda>()>,
            {10018874: defaultdict(list,
                         {('harry',
                           'hermione'): ["`` What 's ccc_Normally_Ron_ccc doing here ? '' `` Not like we had much choice , did we , ccc_Harry_Potter_ccc ? ''"],
                          ('hermione',
                           'ron'): ["`` Certainly , '' `` Well , at least let me help you fetch them . ''", "`` Listen , '' `` ccc_Normally_Ron_ccc 're a complete bastard , ccc_Normally_Ron_ccc know that ? ''"]}),
             10028441: defaultdict(list,
                         {('ginny',
                           'harry'): ["`` Thanks mate , '' `` ccc_Harry_murmured_ccc ! ''", '`` ccc_Lestrange_ccc came into town the other way , from the station . ccc_Hermione_ccc hit ccc_Harry_murmured_ccc with a blasting curse that sent ccc_Harry_murmured_ccc through the wall of Flourish and Blots . Through the wall . Have you ever gone through a wall ? \'\' `` ccc_Ginny_ccc jumped 

In [28]:
len(os.listdir(quote_attribution_output_dirpath))

5249

# Extract paragraph features from extracted character assertions

In [12]:
import os

assertions_dirpath = '/usr0/home/mamille2/erebor/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/output/assertion_extraction' # misty
len(os.listdir(assertions_dirpath))

833

In [15]:
import json

assertions = {}
for fname in sorted(os.listdir(assertions_dirpath))[:1]:
    
    fic_id = int(fname.split('.')[0])
    
    with open(os.path.join(assertions_dirpath, fname)) as f:
        assertions[fic_id] = json.load(f)
        
assertions[fic_id].keys()

dict_keys(['($_summer)', '($_the_whole_``_Harry)', '($_Occulmency_,_as_Snape)', '($_Sirius)', '($_Remus)', '($_Voldemort)', '($_the_Ministry)', '($_Dumbledore)', '($_Emmett)', '($_R.A.B.)', '($_Draco_Malfoy)', '($_the_Death_Eaters)', '($_Jasper)', '($_a_Death_Eater)', '($_Luna)', '($_Neville)', '($_Theodore_Nott)', '($_Hogwarts)', '($_The_Death_Eaters)', '($_the_Order)', '($_to_the_Dursleys)', '($_The_Order)', '($_Illyantia)', '($_Carlisle)', '($_this_Edward)', '($_Esme)', '($_that_night)', '($_Carlilse)', '($_the_day)', '($_Alice)', '($_Rosalie)', '($_Mike_Newton)'])

In [18]:
chars = ['harry', 'ron', 'hermione', 'draco', 'ginny']

In [19]:
matches = [charname for charname in assertions[fic_id] if any([c in charname.lower() for c in chars])]
matches

['($_the_whole_``_Harry)', '($_Draco_Malfoy)']

In [20]:
assertions[fic_id][matches[0]]

['Harry ($_the_whole_``_Harry) smiled thinking about the events that happened during his ($_the_whole_``_Harry) sixth year , some were hard and others were so easy to accept ; once Draco and he ($_the_whole_``_Harry) got past their mutual hatred of each other . \n',
 " `` I ($_the_whole_``_Harry) never told Sirius_and_Remus ($_Sirius) Remus ($_Remus) about the conversation with Dumbledore ($_Dumbledore) after the battle , except the prophesy . I ($_the_whole_``_Harry) knew they would feel guilt for not being there to help me ($_the_whole_``_Harry) when Dumbledore ($_Dumbledore) manipulated my ($_the_whole_``_Harry) life for his ($_Dumbledore) own goals . I ($_the_whole_``_Harry) knew that if they ever found out that they might confront Dumbledore ($_Dumbledore) and I ($_the_whole_``_Harry) was worried about the outcome of that conversation . Sirius ($_Sirius) was an escaped convict and Remus ($_Remus) was a werewolf , Dumbledore ($_Dumbledore) could make their lives hell if he ($_Dumbl

# Assemble processed fics from pipeline for temporary dataset

In [2]:
import os

base_dirpath = '/usr0/home/mamille2/erebor/fanfiction-project/data/ao3/harrypotter' # misty
splits_dirpath = '/usr0/home/mamille2/erebor/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k_splits/' # misty

all_fic_ids = [int(fname[:-4]) for fname in os.listdir(os.path.join(base_dirpath, 'emnlp_dataset_6k', 'fics'))]
len(all_fic_ids)

5594

In [3]:
done_fic_ids = [int(fname.split('.')[0]) for fname in os.listdir(os.path.join(base_dirpath, 'emnlp_dataset_6k', 'output', 'char_coref_chars'))]
print(len(done_fic_ids))

833


[9463769,
 10808058,
 15594333,
 10064609,
 7361353,
 12303831,
 13108605,
 16441577,
 445263,
 761047]

In [8]:
partial_dirpath = os.path.join(base_dirpath, 'emnlp_dataset_6k_partial')
# os.mkdir(partial_dirpath)
# os.mkdir(os.path.join(partial_dirpath, 'output'))
os.mkdir(os.path.join(partial_dirpath, 'output', 'char_coref_chars'))
os.mkdir(os.path.join(partial_dirpath, 'output', 'char_coref_stories'))
os.mkdir(os.path.join(partial_dirpath, 'output', 'assertion_extraction'))

In [9]:
import shutil

for i in range(24):
    
    for dirname in ['char_coref_chars', 'char_coref_stories', 'assertion_extraction']:
        from_dirpath = os.path.join(splits_dirpath, str(i), 'output', dirname)
        to_dirpath = os.path.join(partial_dirpath, 'output', dirname)
        
        for fname in os.listdir(from_dirpath):
            shutil.copy(os.path.join(from_dirpath, fname), os.path.join(to_dirpath, fname))
            
len(os.listdir(from_dirpath))

0

In [10]:
from_dirpath

'/usr0/home/mamille2/erebor/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k_splits/23/output/assertion_extraction'

# Separate fics into done, 100-fic chunks for parallelizing pipeline

In [7]:
import os

data_dirpath = '/usr0/home/mamille2/erebor/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/' # misty

all_fic_ids = [int(fname[:-4]) for fname in os.listdir(os.path.join(data_dirpath, 'fics'))]
len(all_fic_ids)

5594

In [8]:
done_fic_ids = [int(fname.split('.')[0]) for fname in os.listdir(os.path.join(data_dirpath, 'output', 'char_coref_chars'))]
print(len(done_fic_ids))
done_fic_ids[:10]

833


[9463769,
 10808058,
 15594333,
 10064609,
 7361353,
 12303831,
 13108605,
 16441577,
 445263,
 761047]

In [9]:
not_done_fic_ids = sorted([fic_id for fic_id in all_fic_ids if not fic_id in done_fic_ids])
len(not_done_fic_ids)

4761

In [10]:
# Separate into sections of 200 to do "multiprocessing"

n_in_sections = 200
sections = {0: []}
section_ctr = 0

for i, fic_id in enumerate(not_done_fic_ids):
    if i > 0 and i % 200 == 0:
        section_ctr += 1
        sections[section_ctr] = []
        
    sections[section_ctr].append(fic_id)

In [11]:
len(sections)

24

In [19]:
from tqdm import tqdm_notebook as tqdm
import shutil

for i in tqdm(sections):
    os.makedirs(os.path.join('/usr0/home/mamille2/erebor/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k_splits', str(i), 'fics'))

    for fic_id in sections[i]:
        from_dirpath = '/usr0/home/mamille2/erebor/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/fics'
        to_dirpath = f'/usr0/home/mamille2/erebor/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k_splits/{i}/fics'

        shutil.copy(os.path.join(from_dirpath, f'{fic_id}.csv'), os.path.join(to_dirpath, f'{fic_id}.csv'))

# Sample 6k rows, Get fic_ids to run the pipeline

In [8]:
# Load existing data (5+ paragraphs with at least one pairing together)

import pandas as pd

# Load data
data = pd.read_pickle('/usr2/mamille2/fanfiction-project/data/features/relationship_prediction.pkl')
len(data)
print(data.columns)

pairings = data['pairing'].unique()
pairings

Index(['fic_id', 'pairing', 'pairing_embedding_embs',
       'pairing_embedding_unigrams', 'relationship', 'selected_relationships',
       'is_romantic', 'is_canon', 'is_mm'],
      dtype='object')


array([('draco', 'harry'), ('hermione', 'ron'), ('ginny', 'harry'),
       ('draco', 'hermione'), ('harry', 'hermione'), ('harry', 'ron')],
      dtype=object)

In [9]:
canon_relationship_map = {
        ('draco', 'harry'): False,
        ('hermione', 'ron'): True,
        ('ginny', 'harry'): True,
        ('draco', 'hermione'): False,
        ('harry', 'hermione'): False,
        ('harry', 'ron'): False,
                         }

relationship_type_map = {
        ('draco', 'harry'): True,
        ('hermione', 'ron'): False,
        ('ginny', 'harry'): False,
        ('draco', 'hermione'): False,
        ('harry', 'hermione'): False,
        ('harry', 'ron'): True,
                         }

data['is_canon'] = [(is_romantic and canon_relationship_map[pairing]) or (not is_romantic and not canon_relationship_map[pairing]) \
                    for is_romantic, pairing in zip(data['is_romantic'], data['pairing'])]

data['is_mm'] = data['pairing'].map(lambda x: relationship_type_map[x])

In [ ]:
# Save out
data.to_pickle('/usr2/mamille2/fanfiction-project/data/features/relationship_prediction.pkl')

In [10]:
fic_ids = sorted(data['fic_id'].unique())
len(fic_ids)

44647

In [12]:
# Create dict of fic_id: chapter_fnames

data_dirpath = '/usr2/scratch/fanfic/ao3_harrypotter_text/stories/'
fnames = os.listdir(data_dirpath)
len(fnames)

from collections import defaultdict

fic_chapters = defaultdict(list)
for fname in tqdm(fnames):
    fic_id = int(fname.split('_')[0])
    fic_chapters[fic_id].append(fname)
    
len(fic_chapters)

179407

In [16]:
# Copy those fic_ids to new directory for pipeline
import os
from tqdm import tqdm_notebook as tqdm

data_dirpath = '/usr2/scratch/fanfic/ao3_harrypotter_text/stories/'
# data_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/fics_paras
out_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/emnlp_dataset/fics'

for fic_id in tqdm(fic_ids):
    chapter_fnames = sorted(fic_chapters[fic_id])
    
    # Combine into 1 fic text
    with open(os.path.join(out_dirpath, f'{fic_id}.csv'), 'w') as out:
        
        # header
        fname = chapter_fnames[0]
        fpath = os.path.join(data_dirpath, fname)
        with open(fpath) as f:
            for line in f:
                out.write(line)
            
        for fname in chapter_fnames[1:]:
            fpath = os.path.join(data_dirpath, fname)
            with open(fpath) as f:
                for line in f:
                    out.write(line)
#                 lines = f.read().splitlines()
#                 for line in lines[1:]:
#                     out.write(line)

In [ ]:
# Sample equal split among pairings

sample = data.sample(frac=1).groupby('pairing').head(1000)
len(sample)

In [ ]:
# Save out sample
sample.to_pickle('/usr2/mamille2/fanfiction-project/data/features/relationship_prediction_sample6k.pkl')

In [ ]:
sampled_fic_ids = sorted(sample['fic_id'].unique())
len(sampled_fic_ids)

In [ ]:
sampled_fic_ids[:10]

In [ ]:
import shutil

for fic_id in tqdm(sampled_fic_ids):
    from_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/emnlp_dataset/fics'
    to_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/fics'
    
    shutil.copy(os.path.join(from_dirpath, f'{fic_id}.csv'), os.path.join(to_dirpath, f'{fic_id}.csv'))